# Trading strategy simulation

In [15]:
import numpy as np

In [16]:
def get_simret(n=100, mu=1.001, sigma=0.01):
    """Create array of n of normally distibuted returns."""
    return np.random.normal(mu, sigma, n)

def get_prices(sim, init_price=100.):
    """Create array of prices from array of returns"""
    return init_price*np.cumprod(sim)

In [17]:
get_prices(get_simret())

array([ 100.68555159,  101.73523242,   99.92496709,  100.03656939,
         99.41571173,   98.40029399,   97.79949995,   97.32100891,
         98.77420601,   97.45339082,   97.24911399,   97.54004631,
         97.62590507,   99.96492307,  101.03679451,   99.89866267,
        100.38544086,  100.42442827,  100.24586119,  100.14295579,
         99.40940698,  101.10328881,  100.84585212,  101.52013885,
        103.32194995,  103.27278993,  106.04569464,  107.47757511,
        108.12091895,  107.58899016,  107.23560809,  107.25849889,
        107.84922355,  105.54712997,  104.91654086,  104.02533274,
        103.3279301 ,  102.89894778,  103.14054125,  104.83464497,
        103.67625499,  103.12987198,  103.20399209,  103.77819388,
        105.16666267,  106.74494429,  105.99050835,  106.36473919,
        106.80600094,  108.14642319,  108.22490573,  109.57039292,
        109.00123252,  108.85095621,  109.74024342,  109.40224277,
        110.50023419,  109.56303089,  110.34494796,  111.20126

In [20]:
def ret_buynhold(get_ret, init_balance=100.):
    """Calculate total return with buy-and-hold strategy.
       
       We assume that the buy happens at the get_ret()[0] price.
    """
    prices = get_prices(get_ret())
    return (prices[-1]-prices[0])/init_balance

In [21]:
ret_buynhold(get_simret)

0.047631731624976366

In [22]:
def ret_contra(get_ret, streak, init_balance=100.):
    """Trade counter trend after the given number of subsequent trending days ('streak')."""
    if (streak < 1):
        return 0.0  # no trades
    sim = get_ret()
    prices = get_prices(get_ret())
    n = len(sim)
    balance = init_balance
    held = 0.0
    for i in range(streak-1, n):
        if max(sim[i-streak+1:i+1]) < 1.0: # streak consequtive losses - buy if not held
            if held == 0:
                balance -= prices[i]
                held += 1
        elif min(sim[i-streak+1:i+1]) > 1.0: # streak consequtive gains - sell if held
            if held == 1:
                balance += prices[i]
                held -= 1
    return (held*prices[-1] + balance - init_balance)/init_balance

In [23]:
ret_contra(get_simret, 3)

0.0083548680756952362

Let's compare average results for both strategies.

In [24]:
def aver(f, repeat=1000):
    s = 0.0
    for i in range(repeat):
        s += f()
    return s/repeat

In [25]:
aver(lambda: ret_buynhold(get_simret)), aver(lambda: ret_contra(get_simret, 3))

(0.10669053455184395, 0.040591435503252796)

It looks like the counter-trend trading is much worse than buy and hold!